# Segmenting and Clustering Neighborhoods in Toronto

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe¶

In [13]:
import pandas as pd
import numpy as np
import requests

In [14]:
w_link= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
rawpage= requests.get(w_link).text

There are other packages and frameworks, like Scrapy. But Beautiful Soup allows you to parse the HTML in a a beautiful way, so that’s what I’m going to use

In [15]:
import bs4 as bs

In [16]:
xml = bs.BeautifulSoup(rawpage,'xml')
table_body = xml.find('table')

With *table* we retrieved the target info:

In [17]:
data = []
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

In [18]:
type(data)

list

In [19]:
labels = ['PostalCode', 'Borough', 'Neighbourhood']
df = pd.DataFrame.from_records(data, columns=labels)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [20]:
mask=(df["Borough"]!="Not assigned") & (df["Neighbourhood"]=="Not assigned")
df.loc[mask, "Neighbourhood"] = df["Borough"]


In [21]:
df.head()

,PostalCode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Clean **Not assigned** and **None** values

In [22]:
df2=df.drop(df[(df["Borough"]=="Not assigned") | (df["Borough"].isnull())].index)

In [23]:
df3 = df2.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
df3 = df3.reset_index()

In [24]:
df3.shape

(103, 3)

In [25]:
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## PART 2

geocoder is not working for this reason I downloaded codes from http://cocl.us/Geospatial_data

In [26]:
df_geospatial=pd.read_csv("http://cocl.us/Geospatial_data")

In [27]:
df_geospatial.shape

(103, 3)

In [28]:
df_geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
df_toronto = df3.join(df_geospatial.set_index('Postal Code'), on='PostalCode')

In [30]:
df_toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
